In [19]:
import os 
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt 
import pylab
from PIL import Image

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
class Augment:
    def __init__(self, config):
        self.mfcc_data = []
        self.bands = 128
        self.frames = 128
        self.counts = {
            "english": 0,
            "hindi": 0,
            "mandarin": 0,
            "tagalog": 0,
            "other": 0
        }
        self.samples = 1000

        self.pitch_shift_count = config.get("pitch_shift_count",0) #should be even
        self.gaussian_noise_count = config.get("gaussian_noise_count",0)
        self.gaussian_noise_stdev = config.get("gaussian_noise_stdev",1) #1 dB
        self.save_npys = config.get("save_npys",False)
        self.save_jpgs = config.get("save_jpgs",True)
        self.duration = config.get("duration", 1)
        self.sr = config.get("sr", 22050)

        self.abs_dir = "/content/drive/My Drive/APS360 Group Project"

        self.splits = ("train", "validation", "test")
        self.labels = ("english", "hindi", "mandarin", "tagalog")

        # where the raw audio files are located
        self.audio_files_dir = "audio_split"
        self.audio_path = os.path.join(self.abs_dir, self.audio_files_dir)

        # where to save the generated dataset
        self.name = self.name_generator()
        self.save_path = os.path.join(self.abs_dir, self.name)

    def create_dataset(self):
        for split in self.splits:
            for label in self.labels:
                sub_dir = ("%s/%s"%(split,label))
                print("Processing %s..."%(sub_dir))
                self.create_mfccs(sub_dir)

    def name_generator(self):
        return("mfcc_dur_%s_psc_%d_gnc_%d_std_%d"%(self.duration,self.pitch_shift_count,self.gaussian_noise_count, self.gaussian_noise_stdev))

    def windows(self,data,window_size):
        start = 0 
        yield start, start + window_size
        start += (window_size / 2)

    def pitch_shifter(self, speech, sr):
        boundary = int(self.pitch_shift_count/2)
        pitch_shifted_mfccs = []
        for i in range(-boundary, boundary+1):
            if i != 0:
                shifted_speech = librosa.effects.pitch_shift(speech, sr, i)
                pitch_shifted_mfccs.append(self.get_MFCC(shifted_speech))
        return pitch_shifted_mfccs

    def gaussian_noiser(self, mfcc):
        np.random.seed(1)
        gaussian_noised_mfccs = []
        for i in range(0, self.gaussian_noise_count):
            noise = np.random.normal(0,self.gaussian_noise_stdev,mfcc.shape)
            gaussian_noised_mfccs.append(noise + mfcc)
        return gaussian_noised_mfccs

    def get_MFCC(self,speech):
        mfcc = librosa.feature.melspectrogram(speech) 
        logspec = librosa.amplitude_to_db(mfcc)
        return logspec

    def resize_mfcc(self,mfcc):
        resized_mfcc = librosa.util.fix_length(mfcc, self.bands, axis=1)
        resized_mfcc = np.vstack((np.zeros((0, self.bands)), resized_mfcc))
        return resized_mfcc
        
    def save_data(self,data,sub_dir, name_only):
        # data = self.resize_mfcc(data)

        elems = sub_dir.split('/')

        if elems[1] in self.counts.keys():
            num = self.counts[elems[1]]
            print(num)
            self.counts[elems[1]] += 1
            if num >= self.samples:
                return True

        else:
            num = self.counts["other"]
            self.counts["other"] += 1 
            if num >= self.samples:
                return True
                
        if self.save_npys: # deprecated, but still usable
            new_filename_npy = name_only + "_" + str(num) + '.npy'
            path_npy = [self.save_path,sub_dir,new_filename_npy]
            save_path_npy = os.path.join(*path_npy)
            try:
                np.save(save_path_npy,data)
            except FileNotFoundError:
                save_dir = os.path.join(self.save_path, sub_dir)
                os.makedirs(save_dir)
                np.save(save_path_npy,data)

        if self.save_jpgs:
            new_filename_jpg = name_only + "_" + str(num) + '.jpg'
            path_jpg = [self.save_path,sub_dir,new_filename_jpg]
            save_path_jpg = os.path.join(*path_jpg)
            save_dir_path_jpg = os.path.join(self.save_path, sub_dir)
            pylab.axis('off')
            pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
            librosa.display.specshow(data)
            try:
                pylab.savefig(save_path_jpg, bbox_inches=None, pad_inches=0)
                pylab.close()
            except FileNotFoundError:
                save_dir = os.path.join(self.save_path, sub_dir)
                os.makedirs(save_dir)
                pylab.savefig(save_path_jpg, bbox_inches=None, pad_inches=0)
                pylab.close()

        return True 
    
    def uniform_clip_split(self, sub_dir, file_name):
        path = [self.audio_path,sub_dir,file_name] 
        file_path = os.path.join(*path)
        print(file_path)

        # check if this is train, val, or test
        split = sub_dir.split("/")[0]

        name_only = file_name.split(".mp3")[0]
        speech,s = librosa.load(file_path, self.sr)
        num_frames = int(self.duration*s)
        for i in range(0, int(len(speech)/num_frames)):
            start = i*num_frames
            end = start + num_frames 
            if (len(speech[start:end]) == num_frames):
                clip = speech[start:end]

                # generate and save unaltered MFCC
                raw_mfcc = self.get_MFCC(clip)
                print("Shape:",raw_mfcc.shape)
                self.save_data(raw_mfcc,sub_dir,name_only)

                if split == "train":
                    # generate and save noisy, unshifted MFCCs (only for training)
                    noisy_mfccs = self.gaussian_noiser(raw_mfcc)
                    for i, noisy_mfcc in enumerate(noisy_mfccs):
                        self.save_data(noisy_mfcc,sub_dir,(name_only+"_noisy%d"%(i)))

                    # generate and save pitch-shifted MFCCs - both noisy and non-noisy
                    shifted_mfccs = self.pitch_shifter(speech,self.sr)
                    lowest_shift = -1*int(self.pitch_shift_count/2)
                    for i, shifted_mfcc in enumerate(shifted_mfccs):
                        shift = lowest_shift + i
                        shifted_name = name_only+("_shifted%d"%shift)
                        self.save_data(shifted_mfcc,sub_dir,shifted_name)

                        noisy_shifted_mfccs = self.gaussian_noiser(shifted_mfcc)
                        for j, noisy_shifted_mfcc in enumerate(noisy_shifted_mfccs):
                            noisy_shifted_name = shifted_name + ("_noisy%d"%j)
                            self.save_data(noisy_shifted_mfcc,  sub_dir, shifted_name)

    def create_mfccs(self, sub_dir):
        for file_name in os.listdir(os.path.join(self.audio_path,sub_dir)):
            print(file_name)
            self.uniform_clip_split(sub_dir,file_name)
        return True

In [ ]:
config = {
    "pitch_shift_count": 4,
    "gaussian_noise_count": 2,
    "gaussian_noise_stdev": 1,
    "save_jpgs": True,
    "save_npys": False,
    "duration": 1,
    "sr": 22050
}

data = Augment(config)

data.create_dataset()

Processing train/english...
english105.mp3
/content/drive/My Drive/APS360 Group Project/audio_split/train/english/english105.mp3
Shape: (128, 44)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Shape: (128, 44)
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
Shape: (128, 44)
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
Shape: (128, 44)
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
Shape: (128, 44)
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
Shape: (128, 44)
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
Shape: (128, 44)
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
Shape: (128, 44)
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
Shape: (128, 44)
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
Shape: (128, 44)
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
Shape: (128, 44)
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
Shape: (128, 44)
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
Shape: (128, 44)
180
181
182
183
184
185
186
187
188
189


In [18]:
%%shell

pwd
!cd "drive/My Drive/APS360 Group Project"
!pwd

/content
/content
